# **Getting to know Llama 2: Everything you need to start building**
Our goal in this session is to provide a guided tour of Llama 2, including understanding different Llama 2 models, how and where to access them, Generative AI and Chatbot architectures, prompt engineering, RAG (Retrieval Augmented Generation), Fine-tuning and more. All this is implemented with a starter code for you to take it and use it in your Llama 2 projects.

## **0 - Prerequisites**
* Basic understanding of Large Language Models
* Basic understanding of Python

In [1]:
# presentation layer code

import base64

import ipywidgets as widgets
from IPython.display import display, Image, Markdown


def mm(graph):
    graphbytes = graph.encode("ascii")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))


def md(t):
    display(Markdown(t))


def genai_app_arch():
    mm("""
    flowchart TD
        A[Users] --> B(Applications e.g. mobile, web)
        B --> |Hosted API|C(Platforms e.g. Custom, HuggingFace, Replicate)
        B -- optional --> E(Frameworks e.g. LangChain)
        C-->|User Input|D[Llama 2]
        D-->|Model Output|C
        E --> C
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def rag_arch():
    mm("""
    flowchart TD
        A[User Prompts] --> B(Frameworks e.g. LangChain)
        B <--> |Database, Docs, XLS|C[fa:fa-database External Data]
        B -->|API|D[Llama 2]
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
  """)


def llama2_family():
    mm("""
    graph LR;
        llama-2 --> llama-2-7b
        llama-2 --> llama-2-13b
        llama-2 --> llama-2-70b
        llama-2-7b --> llama-2-7b-chat
        llama-2-13b --> llama-2-13b-chat
        llama-2-70b --> llama-2-70b-chat
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def apps_and_llms():
    mm("""
    graph LR;
        users --> apps
        apps --> frameworks
        frameworks --> platforms
        platforms --> Llama 2
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


# Create a text widget
API_KEY = widgets.Password(
    value='',
    placeholder='',
    description='API_KEY:',
    disabled=False
)


def bot_arch():
    mm("""
    graph LR;
        user --> prompt
        prompt --> i_safety
        i_safety --> context
        context --> Llama_2
        Llama_2 --> output
        output --> o_safety
        i_safety --> memory
        o_safety --> memory
        memory --> context
        o_safety --> user
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def fine_tuned_arch():
    mm("""
    graph LR;
        Custom_Dataset --> Pre-trained_Llama
        Pre-trained_Llama --> Fine-tuned_Llama
        Fine-tuned_Llama --> RLHF
        RLHF --> |Loss:Cross-Entropy|Fine-tuned_Llama
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def load_data_faiss_arch():
    mm("""
    graph LR;
        documents --> textsplitter
        textsplitter --> embeddings
        embeddings --> vectorstore
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def mem_context():
    mm("""
    graph LR
        context(text)
        user_prompt --> context
        instruction --> context
        examples --> context
        memory --> context
        context --> tokenizer
        tokenizer --> embeddings
        embeddings --> LLM
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


## **1 - Understanding Llama 2**

### **1.1 - What is Llama 2?**
* State of the art (SOTA), Open Source LLM
* 7B, 13B, 70B
* Pretrained + Chat
* Choosing model: Size, Quality, Cost, Speed
* [Research paper](https://ai.meta.com/research/publications/llama-2-open-foundation-and-fine-tuned-chat-models/)
* [Responsible use guide](https://ai.meta.com/llama/responsible-use-guide/)

In [2]:
llama2_family()

### **1.2 - Accessing Llama 2**
* Download + Self Host (on-premise)
* Hosted API Platform (e.g. [Replicate](https://replicate.com/meta))
* Hosted Container Platform (e.g. [Azure](https://techcommunity.microsoft.com/t5/ai-machine-learning-blog/introducing-llama-2-on-azure/ba-p/3881233), [AWS](https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/), [GCP](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/139))

### **1.3 - Use Cases of Llama 2**
* Content Generation
* Chatbots
* Summarization
* Programming (e.g. Code Llama)
* and many more...

## **2 - Using Llama 2**
In this notebook, we are going to access [Llama 13b chat model](https://replicate.com/meta/llama-2-13b-chat) using hosted API from Replicate.

### **2.1 - Install dependencies**

In [3]:
# Install dependencies and initialize
!pip install -r requirements.txt

In [4]:
# model url on Replicate platform that we will use for inferencing
# We will use llama 2 13b chat model hosted on replicate server ()agent_name = "llama-2-13b-chat"
model_name = "llama-2-13b-chat"
llama2_13b = f"meta/{model_name}:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

In [5]:
# We will use Replicate hosted cloud environment
# Obtain Replicate API key → https://replicate.com/account/api-tokens)

# enter your replicate api token
import os
import logging
from chrisbase.io import read_or
from getpass import getpass

REPLICATE_API_TOKEN = read_or(".replicate_api_token") or getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

from chrisbase.io import LoggingFormat
from chrisbase.data import JobTimer, ProjectEnv, CommonArguments

logging.getLogger("IPKernelApp").setLevel(logging.INFO)
logger = logging.getLogger(__name__)
args = CommonArguments(
    env=ProjectEnv(
        project="LLM-based",
        job_name="Llama 2 13b Chat",
        msg_level=logging.INFO,
        msg_format=LoggingFormat.CHECK_12,
    )
)
args.dataframe()

,CommonArguments,value
0,tag,None
1,env.project,LLM-based
2,env.job_name,Llama 2 13b Chat
3,env.job_version,None
4,env.hostname,ChrisBookPro.local
5,env.hostaddr,172.20.10.5
6,env.time_stamp,0426.180439
7,env.python_path,/Users/chris/miniforge3/envs/LLM-based/bin/python3.11
8,env.current_dir,/Users/chris/proj/LLM-based
9,env.current_file,/Users/chris/proj/LLM-based/Getting_to_know_Llama_2.ipynb


In [6]:
# we will use replicate's hosted api
import replicate


# text completion with input prompt
def Completion(prompt):
    output = replicate.run(
        llama2_13b,
        input={"prompt": prompt,
               "max_new_tokens": 1000}
    )
    return "".join(output)


# chat completion with input prompt and system prompt
def ChatCompletion(prompt, system_prompt=None):
    output = replicate.run(
        llama2_13b,
        input={"system_prompt": system_prompt,
               "prompt": prompt,
               "max_new_tokens": 1000}
    )
    return "".join(output)

### **2.2 - Basic completion**

In [7]:
with JobTimer("2.2 - Basic completion", rt=1, rb=1, rc='=', verbose=1):
    output = Completion(prompt="The typical color of a llama is: ")
    logger.info(f"{model_name}: {output}")

[04.26 18:04:39] ┇ INFO     ┇ chrisbase.data ┇ =========================================================================================================================================
[04.26 18:04:39] ┇ INFO     ┇ chrisbase.data ┇ [INIT] 2.2 - Basic completion
[04.26 18:04:39] ┇ INFO     ┇ chrisbase.data ┇ =========================================================================================================================================
[04.26 18:04:40] ┇ INFO     ┇        httpx ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.26 18:04:41] ┇ INFO     ┇        httpx ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.26 18:04:41] ┇ INFO     ┇        httpx ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/pv9f1bjeqhrgp0cf36karwz2tr "HTTP/1.1 200 OK"
[04.26 18:04:42] ┇ INFO     ┇        httpx ┇ HTTP Request: GET 

### **2.3 - System prompts**

In [8]:
with JobTimer("2.3 - System prompts", rt=1, rb=1, rc='=', verbose=1):
    output = ChatCompletion(
        prompt="The typical color of a llama is: ",
        system_prompt="respond with only one word"
    )
    logger.info(f"{model_name}: {output}")

[04.26 18:04:43] ┇ INFO     ┇ chrisbase.data ┇ =========================================================================================================================================
[04.26 18:04:43] ┇ INFO     ┇ chrisbase.data ┇ [INIT] 2.3 - System prompts
[04.26 18:04:43] ┇ INFO     ┇ chrisbase.data ┇ =========================================================================================================================================
[04.26 18:04:43] ┇ INFO     ┇        httpx ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.26 18:04:43] ┇ INFO     ┇        httpx ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.26 18:04:44] ┇ INFO     ┇        httpx ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/ce4tse2t9xrgg0cf36kav17c08 "HTTP/1.1 200 OK"
[04.26 18:04:44] ┇ INFO     ┇     __main__ ┇ llama-2-13b-chat:  G

### **2.4 - Response formats**
* Can support different formatted outputs e.g. text, JSON, etc.

In [9]:
with JobTimer("2.4 - Response formats", rt=1, rb=1, rc='=', verbose=1):
    output = ChatCompletion(
        prompt="The typical color of a llama is: ",
        system_prompt="response in json format"
    )
    logger.info(f"{model_name}: {output}")

[04.26 18:04:45] ┇ INFO     ┇ chrisbase.data ┇ =========================================================================================================================================
[04.26 18:04:45] ┇ INFO     ┇ chrisbase.data ┇ [INIT] 2.4 - Response formats
[04.26 18:04:45] ┇ INFO     ┇ chrisbase.data ┇ =========================================================================================================================================
[04.26 18:04:45] ┇ INFO     ┇        httpx ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.26 18:04:46] ┇ INFO     ┇        httpx ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.26 18:04:46] ┇ INFO     ┇        httpx ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/6xgg3t32z9rgp0cf36k8ayqg20 "HTTP/1.1 200 OK"
[04.26 18:04:46] ┇ INFO     ┇     __main__ ┇ llama-2-13b-chat: 